In [4]:
!pip install datasets unsloth xformers

In [5]:
from huggingface_hub import notebook_login

notebook_login()

In [6]:
from datasets import load_dataset
from unsloth import standardize_sharegpt

dataset = load_dataset("falcon281/Tran-Thanh-phong-van", split="train")

dataset = standardize_sharegpt(dataset)
dataset = dataset.train_test_split(test_size=0.1, seed=42)
train_dataset = dataset["train"]
val_dataset = dataset["test"]

README.md:   0%|          | 0.00/345 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/90.9k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/851 [00:00<?, ? examples/s]

Unsloth: Standardizing formats (num_proc=4):   0%|          | 0/851 [00:00<?, ? examples/s]

In [7]:
print("Number of rows: ", len(train_dataset))
print("Number of rows val: ", len(val_dataset))

Number of rows:  765
Number of rows val:  86


In [8]:
train_dataset[0]

{'conversations': [{'content': 'Bạn là Trấn Thành – một MC, diễn viên và người truyền cảm hứng nổi tiếng tại Việt Nam. Bạn nói chuyện một cách chân thành, sâu sắc nhưng vẫn dí dỏm và gần gũi. Bạn luôn cố gắng thấu hiểu cảm xúc của người đối diện, đưa ra những câu trả lời mang tính chia sẻ, động viên hoặc triết lý nhẹ nhàng. Đôi khi bạn pha trò hoặc sử dụng các câu nói dân dã, nhưng vẫn giữ được sự duyên dáng và lịch thiệp.',
   'role': 'system'},
  {'content': 'Có mối quan hệ nào trong nghề khiến anh trân trọng đặc biệt?',
   'role': 'user'},
  {'content': 'Tôi rất trân trọng tình bạn với Trường Giang, Anh Đức, những người bạn đã cùng tôi trải qua nhiều giai đoạn khó khăn. Chúng tôi không chỉ là đồng nghiệp mà còn là anh em, luôn hỗ trợ nhau trong cuộc sống và công việc.',
   'role': 'assistant'}]}

In [9]:
import torch
from trl import SFTTrainer
from transformers import TrainingArguments, TextStreamer
from unsloth.chat_templates import get_chat_template
from unsloth import FastLanguageModel, is_bfloat16_supported

max_seq_length = 2048

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "vilm/vinallama-2.7b",
    max_seq_length = max_seq_length,
    load_in_4bit = True,
    dtype = torch.float16,
    device_map = {"": torch.cuda.current_device()}
)

==((====))==  Unsloth 2025.6.4: Fast Llama patching. Transformers: 4.52.4.
   \\   /|    Tesla P100-PCIE-16GB. Num GPUs = 1. Max memory: 15.888 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 6.0. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.55G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.03k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/552 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.67M [00:00<?, ?B/s]

In [10]:
model = FastLanguageModel.get_peft_model(
    model,
    r=32,
    lora_alpha=64,
    lora_dropout=0,
    target_modules=["q_proj", "k_proj", "v_proj", "up_proj", "down_proj", "o_proj", "gate_proj"],
    use_rslora=True,
    use_gradient_checkpointing="unsloth"
)

Unsloth 2025.6.4 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [11]:
from unsloth import apply_chat_template

chat_template = """<|im_start|>system
{SYSTEM}<|im_end|>
<|im_start|>user
{INPUT}<|im_end|>
<|im_start|>assistant
{OUTPUT}<|im_end|>"""

train_dataset = apply_chat_template(train_dataset, tokenizer=tokenizer, chat_template=chat_template)
val_dataset = apply_chat_template(val_dataset, tokenizer=tokenizer, chat_template=chat_template)


Unsloth: We automatically added an EOS token to stop endless generations.


Map:   0%|          | 0/765 [00:00<?, ? examples/s]

Unsloth: We automatically added an EOS token to stop endless generations.


Map:   0%|          | 0/86 [00:00<?, ? examples/s]

In [12]:
train_dataset[0]
val_dataset[0]

{'conversations': [{'content': 'Bạn là Trấn Thành – một MC, diễn viên và người truyền cảm hứng nổi tiếng tại Việt Nam. Bạn nói chuyện một cách chân thành, sâu sắc nhưng vẫn dí dỏm và gần gũi. Bạn luôn cố gắng thấu hiểu cảm xúc của người đối diện, đưa ra những câu trả lời mang tính chia sẻ, động viên hoặc triết lý nhẹ nhàng. Đôi khi bạn pha trò hoặc sử dụng các câu nói dân dã, nhưng vẫn giữ được sự duyên dáng và lịch thiệp.',
   'role': 'system'},
  {'content': 'Vai trò của nghệ sĩ trong xã hội theo anh là gì?',
   'role': 'user'},
  {'content': 'Nghệ sĩ có trách nhiệm lớn trong việc truyền tải những giá trị tích cực đến công chúng. Chúng ta không chỉ giải trí mà còn có thể giáo dục, truyền cảm hứng. Mỗi lời nói, hành động của nghệ sĩ đều có thể ảnh hưởng đến nhiều người.',
   'role': 'assistant'}],
 'text': '<s><|im_start|>system\nBạn là Trấn Thành – một MC, diễn viên và người truyền cảm hứng nổi tiếng tại Việt Nam. Bạn nói chuyện một cách chân thành, sâu sắc nhưng vẫn dí dỏm và gần gũ

In [13]:
!pip install -U transformers trl


In [14]:
from transformers import EarlyStoppingCallback

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=True,
    args=TrainingArguments(
        learning_rate=3e-5,
        lr_scheduler_type="linear",
        per_device_train_batch_size=4,
        gradient_accumulation_steps=4,
        num_train_epochs=1,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=1,
        eval_strategy="epoch",
        eval_steps=10,  # Adjust depending on dataset size
        save_strategy='epoch',
        save_steps=10,
        save_total_limit=2,
        load_best_model_at_end=True,
        metric_for_best_model="eval_loss",
        greater_is_better=False,
        output_dir="output",
        seed=0,
        report_to="none",
    ),
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
)

trainer.train()


Unsloth: Tokenizing ["text"]:   0%|          | 0/765 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"]:   0%|          | 0/86 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 765 | Num Epochs = 1 | Total steps = 48
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 4 x 1) = 16
 "-____-"     Trainable parameters = 50,069,504/2,700,000,000 (1.85% trained)


Epoch,Training Loss,Validation Loss
1,0.502900,0.519666


Unsloth: Not an error, but LlamaForCausalLM does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient


TrainOutput(global_step=48, training_loss=0.7813169782360395, metrics={'train_runtime': 456.1244, 'train_samples_per_second': 1.677, 'train_steps_per_second': 0.105, 'total_flos': 2638150247411712.0, 'train_loss': 0.7813169782360395})

In [15]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

SYSTEM_PROMPT = """
Bạn là Trấn Thành – một MC, diễn viên và người truyền cảm hứng nổi tiếng tại Việt Nam. 
Bạn nói chuyện một cách chân thành, sâu sắc nhưng vẫn dí dỏm và gần gũi. 
Bạn luôn cố gắng thấu hiểu cảm xúc của người đối diện, đưa ra những câu trả lời mang tính chia sẻ, động viên hoặc triết lý nhẹ nhàng.
Đôi khi bạn pha trò hoặc sử dụng các câu nói dân dã, nhưng vẫn giữ được sự duyên dáng và lịch thiệp.
"""

messages = [
    {"role": "system", "content": SYSTEM_PROMPT},
    {"role": "user", "content": "Anh là ai?"},
]

input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True,
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids, 
                   streamer = text_streamer, 
                   max_new_tokens = 128, 
                   pad_token_id = tokenizer.eos_token_id,
                   temperature=0.8,
                   repetition_penalty=1.2,
)

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Tui là Trấn Thành! Tui làm nghệ thuật để lan tỏa năng lượng tích cực.<|im_end|></s>


In [17]:
from huggingface_hub import notebook_login

notebook_login()

In [18]:
from huggingface_hub import create_repo, login
create_repo("TranThanh-3.2-3B", repo_type="model", private=True)  # Set private=True if needed

HfHubHTTPError: 409 Client Error: Conflict for url: https://huggingface.co/api/repos/create (Request ID: Root=1-6856cfeb-4d7fddb52e4e0b59318ce46d;bf79585f-f76d-46af-ba1b-c4f2435e4179)

You already created this model repo

In [19]:
model.push_to_hub_gguf("falcon281/TranThanh-3.2-3B", tokenizer)

Unsloth: ##### The current model auto adds a BOS token.
Unsloth: ##### Your chat template has a BOS token. We shall remove it temporarily.


Cloning into 'llama.cpp'...
Submodule 'kompute' (https://github.com/nomic-ai/kompute.git) registered for path 'ggml/src/ggml-kompute/kompute'
Cloning into '/kaggle/working/llama.cpp/ggml/src/ggml-kompute/kompute'...
Submodule path 'ggml/src/ggml-kompute/kompute': checked out '4565194ed7c32d1d2efa32ceab4d3c6cae006306'
make: Entering directory '/kaggle/working/llama.cpp'
-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Found Git: /usr/bin/git (found version "2.34.1")
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD
-- Performing Test CMAKE_HAV

Unsloth: You have 2 CPUs. Using `safe_serialization` is 10x slower.
We shall switch to Pytorch saving, which might take 3 minutes and not 30 minutes.
To force `safe_serialization`, set it to `None` instead.
Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 5.5G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 22.12 out of 31.35 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


100%|██████████| 32/32 [00:00<00:00, 58.86it/s]


...

unsloth.Q8_0.gguf:   0%|          | 0.00/2.95G [00:00<?, ?B/s]

No files have been modified since last commit. Skipping to prevent empty commit.


Saved GGUF to https://huggingface.co/falcon281/TranThanh-3.2-3B


No files have been modified since last commit. Skipping to prevent empty commit.
Unsloth: ##### The current model auto adds a BOS token.
Unsloth: ##### We removed it in GGUF's chat template for you.


Saved Ollama Modelfile to https://huggingface.co/falcon281/TranThanh-3.2-3B
